# Weather Changes Over Time (Consuming APIs)

## Task
Utilize the following two APIs to evaluate the changes in temperature for your birthday and city you were born in:
* [geocode.xyz](geocode.xyz)
* [open-meteo.com](open-meteo.com)


In [132]:
from datetime import date, datetime, timedelta

import requests
import pandas as pd
import numpy as np

import seaborn as sns
from matplotlib import pyplot as plt
from matplotlib.ticker import FormatStrFormatter

%matplotlib inline

sns.set_palette('muted')
sns.set_style('white')

## Question 1: Geocoding A Location

> Indented block


Write a function that utilizes the [geocode.xyz](geocode.xyz) API to take in a city and country and return the latitude and longitude of that location

```
def geocode_location(city, country):
    # your code
    return latitude, longitude
```

For example:
```
geocode_location("london", "united kingdom")
```
should return
```
(51.49742, -0.11534)
```

In [134]:
# write your solution here
def geocode_location(city, country):
    location = requests.get(f"https://geocode.xyz/{city},{country}?json=1")
    location = location.json()
    latitude = location['latt']
    longitude = location['longt']
    return latitude,longitude


latitude, longitude = geocode_location('london', 'united kingdom')
print(f'({latitude}, {longitude})')

(51.51412, -0.11460)


## Question 2: Determining Weather Using Date and Location
Write a function that utilizes the [open-meteo.com](https://open-meteo.com/) API to take in a date and location and return the 15-day average high and low for the following measurements from the years 1960 to 2022:
* `precipitation_sum` (mm): Sum of daily precipitation (including rain, showers and snowfall)
* `temperature_2m_max` (°F): Maximum daily air temperature at 2 meters above ground
* `temperature_2m_min` (°F): Minimum daily air temperature at 2 meters above ground
* `apparent_temperature_max` (°F): Maximum daily apparent temperature
* `apparent_temperature_min` (°F): Min daily apparent temperature

The function should return the above measurements in a dictionary object with the keys being `"precipitation_sum"`, `"temperature_2m_max"`, etc. and the values being the 15-day average of the measurements. The 15-day average should be the 7 days preceeding the input date, the input date, and the 7 days following the input date. This will give the results more statistical validity by increasing the number of samples for each measurement.

The `get_historical_weather_measurements()` function will include a few different sub-tasks:

### Create API Request in Postman/Insomnia
Use the following [API Documentation](https://open-meteo.com/en/docs/historical-weather-api#api-documentation) page to create an API request that responds with a JSON object of the measurements listed above. Make sure the response object meets the following:
* Temperatue measurements are in Fahrenheit
* Precipitation measurements are in inches
* Measurements are daily aggregations
* 15 day range of weather
* JSON format

Before writing any code, use Postman/Insomnia to create the API request and take a screenshot of the request/response.

#### HINT: How to insert a screenshot
1. Take a screenshot and save it to a file.
2. Create a markdown cell in your notebook.
3. Insert the image into the markdown cell.
  * Option 1: Drag the image file into the markdown cell. This embeds the image data directly into your Jupyter notebook.
  * Option 2: Move the file into the directory with your notebook file (inside the project folder that you will submit). Add markdown to insert the image: `![alternative text](path-to-image)`

### Create API Request Function
Convert the request above into a function. The function should take in `latitude`, `longitude`, and `date` as its arguments and return the API response, if successful.

```
def get_api_response(latitude, longitude, date):
    # your code

```

Use the helper function below for creating a 15-day date range

In [136]:
# helper function for 15-day date range
def get_date_range(birthday):
    return {'start_date': birthday - timedelta(days=7),
            'end_date': birthday + timedelta(days=7)
           }

In [153]:
# write your solution here
def get_api_response(latitude, longitude, birthday):
    url = 'https://api.open-meteo.com/v1/archive'
    date_range = get_date_range(birthday)

    params = {'latitude': latitude,
             'longitude': longitude,
             'start_date': date_range['start_date'],
             'end_date': date_range['end_date'],
             'daily': ['temperature_2m_max','temperature_2m_min','apparent_temperature_max','apparent_temperature_min','precipitation_sum'],
             'timezone': 'auto',
             'temperatue_unit': 'farenheit',
             'precipitation_unit': 'inch'}
    r = requests.get(url, params=params).json()
    return r


### Iterating Over Each Year
Write a function that iterates through each year between 1960 and 2022 and returns the corresponding JSON response objects containing weather data for each year in a python list.

```
def get_all_response_objects(start_year, end_year):
    # your code
```


In [156]:
# write your solution here
def get_all_response_objects(latitude, longitude, birthday):
    responses = []
    for year in range(1960, 2023):
        request_date = date(year, birthday.month, birthday.day)
        response = get_api_response(latitude, longitude, request_date)
        responses.append(response)
    
    return responses    



In [158]:
responses = get_all_response_objects(51.51413, -0.11534, date(2000, 12, 1))

### Parse JSON Responses & Aggregate Data
Write a function that takes the list of JSON response objects from the previous function in as a parameter and returns a pandas DataFrame containing the original input date, the measurements, and the average across the 15 days.
```
def parse_json_response(responses):
    # your code
```


In [160]:
# write your solution here
def parse_json_response(responses):
    df = pd.DataFrame(responses)
    df = df.join(pd.json_normalize(df['daiily']))\
           .drop(columns=['daily'])
    
    # Get average of measurements
    measures = ['temperature_2m_max', 'temperature_2m_min',
                'apparent_temperature_max', 'apparent_temperature_min',
                'precipitation_sum']
    
    for col in measures:
        df[f"{col}_mean"] = df[col].apply(np.mean)
        
    # add year column
    df['year'] = df['time'].apply(lambda l: l[0][:4])
    
    return df[['year'] + [measure + '_mean' for measure in measures]]

In [151]:
df = parse_json_response(responses)

KeyError: 'daily'

### Combining the Functions
Write a function that combines all the above functions into one, where the user inputs their birthday and location and the function returns the pandas DataFrame containing the summary of measurements.

```
def get_data():
    birthday = get_user_birthday()
    location = get_user_city_of_birth()
    # your code here
```

The helper functions below can be used for birthday and location input information:

In [ ]:
# helper functions
def get_user_birthday():
    birthday = input('Enter your birthday (MM/DD/YYYY): ')
    return datetime.strptime(birthday, "%m/%d/%Y")

def get_user_city_of_birth():
    city = input('Enter the city you were born in: ')
    country = input('Enter the country you were born in: ')
    return city , country

In [ ]:
# write your solution here
def get_data():
    birthday = get_user_birthday()
    location = get_user_city_of_birth()
    
    latitude, longitude = geocode_location(*location)
    
    responses = get_all_response_objects(latitude, longitude, birthday)
    
    return parse_json_response(responses)

Run your function and set it equal to `df`:

In [ ]:
df = get_data()

df.head()

## Question 3: Visualizing The Data
Plot the DataFrame using `matplotlib`/`seaborn` and write a short summary of your results.

In [ ]:
# enter your code here

### Results Summary: